# Spacy2 NE

## Table of contents: <br>
-  <a href='#declare_section'>Declare section</a>
### LOC/ORG
-  <a href='#load_location_keywords'> Load location/ORG keywords</a>
-  <a href='#train_location'> Train location/org</a>
-  <a href='#load_location'> Load location/org model </a>
-  <a href='#test_location'> Test location/org</a>
### FACILITY
-  <a href='#load_facility_keywords'> Load FACILITY keywords</a>
-  <a href='#train_facility'> Train facility</a>
-  <a href='#load_facility'> Load location/org model </a>
-  <a href='#test_facility'> Test facility</a>
### LOC/ORG/FACITY
-  <a href='#train_both'> Train loc/org/facility</a>
-  <a href='#load_both'> Load location/org model </a>
-  <a href='#test_both'> Test location/org/facility</a>

### Full Description (unknown test)
-  <a href='#load_desc'> Load full description</a>
-  <a href='#test_desc'> Test full description</a>
*** 

## Declare / load corpus/ multiword tokenizer
<a id='declare_section'></a>

In [2]:
from __future__ import absolute_import, division, print_function

In [3]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import keras
import nltk
import spacy

Using TensorFlow backend.


In [4]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

In [5]:
from ipywidgets import IntProgress
from IPython.display import display

In [6]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [8]:
word_files = ['location_hotels.txt']
word_list = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_list.extend(word_list2)
word_list[0:5]

['At Niman Conceptual Home',
 'Baan Nisarine',
 'Coconut Paradise Holiday Villas',
 'Din Space Bangkok',
 'KK inn south pattaya hotel']

In [9]:
word_files = ['facility.txt']
word_list_fac = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_list_fac.extend(word_list2)
word_list_fac[0:5]

['24-hour Receptionist',
 'Laundry service',
 'Luggage storage',
 'Tours',
 'Parking']

In [10]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [11]:
len(word_list)

17181

In [12]:
word_list = ordered_set(word_list)

In [13]:
word_list_fac = ordered_set(word_list_fac)

In [14]:
len(word_list)

17146

In [15]:
mw_list = []
for x in word_list:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

for x in word_list_fac:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
         
        
mw_list[0:5]

[('At', 'Niman', 'Conceptual', 'Home'),
 ('Baan', 'Nisarine'),
 ('Coconut', 'Paradise', 'Holiday', 'Villas'),
 ('Din', 'Space', 'Bangkok'),
 ('KK', 'inn', 'south', 'pattaya', 'hotel')]

In [16]:
for x in word_list_fac:
    if x == "In-room safe":
        print(x)

In-room safe


In [17]:
for i in mw_list:
    if i == ('M&A','Guesthouse'):
        print('found')
    if i == ('In-room','safe'):
        print('found2')

found
found2


In [18]:
mwe = MWETokenizer(mw_list,separator=' ')

In [19]:
from progressbar import AnimatedMarker, Bar, BouncingBar, Counter, ETA, \
    AdaptiveETA, FileTransferSpeed, FormatLabel, Percentage, \
    ProgressBar, ReverseBar, RotatingMarker, \
    SimpleProgress, Timer, UnknownLength
widgets = ['Running: ', Percentage(), ' ',
               Bar(marker='#',left='[',right=']'),
               ' ', ETA() ]

In [20]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [84]:
# check accuracy
def count_accuracy(TEST_DATA):
    
    total_correct = 0
    total = 0
    total_predict = 0
    pbar = ProgressBar(widgets=widgets, maxval=len(TEST_DATA))
    #f = IntProgress(min=0, max=len(TEST_DATA)) # instantiate the bar
    #display(f) # display the bar
    pbar.start()
    count = 0 
    
    for text, annotations in TEST_DATA:
        doc = nlp2(text )
        #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        
        #f.value += 1
        ## get token,position, type
        answer= [(ent.text, ent.label_) for ent in doc.ents]
        s = 0
        predicted = []
        for t in doc:
            if len(answer) == 0 :
                break
            tup = answer[0]
            if t.ent_type_ == tup[1]:
                if t.text == tup[0]:
                    s += 1
                    ind = t.i 
                    end_ind = t.i + len(t.text)
                    save_tuple = (t.text,ind, end_ind, t.ent_type_)
                    predicted.append(save_tuple)
                    if s == len(answer):
                        break;
        #if len(predicted) > 0:
            #print(text)
            #print(predicted)       
            #print(annotations['entities'])
        
        
        #displacy.render(nlp2(str(text).decode('utf-8')), jupyter=True, style='ent
        all_equal = False
        
         
        if (len(annotations['entities']) == len(predicted)): # check len predicted == annotation
            all_equal = True
             
            for i, p in zip(annotations['entities'],predicted):
                word_text= text[int(i[0]):int(i[1])] # extract text
                 
                #item =  {'start': i[0], 'end': i[1], 'label': i[2]}
                #print(word_text, item)
                if (i[2] == p[3] and # same label
                   p[0] == word_text ):  # same word
                   #and p[1] == int(i[0]) and  p[2] == int(i[1])) :
                    total_correct += 1
                    continue
                else:
                    all_equal = False
                    break
        #if all_equal or  len(annotations['entities']) < len(predicted):
        #    total_correct += 1
        total_predict += len(predicted)
        total += len(annotations['entities'])
        count += 1           
        pbar.update(count)
    print("total %d/%d/%d"%(total_correct,total_predict,total))
    pbar.finish()
    return (total_correct/total_predict, total_correct/total)
                            
                        
                
       

##  Train location/ORG
<a id='train_location'></a>

<a href='#declare_section'>Link to top</a>

Load location training data (corpus) first.

In [22]:
import re
reviews = pd.read_csv("./location.csv")
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)
#reviews =  [ re.sub(r'[_"\-;%()|+&=*%!:#$@\[\]/]',' ', i.decode('utf-8')) for i in reviews.location]
reviews.head()

,location
0,Staying at @ Home Executive Apartment is a goo...
1,Staying at @ Oasis Resort is a good choice whe...
2,Staying at @ T Boutique Hotel is a good choice...
3,Staying at AC Resort is a good choice when you...
4,Staying at Acqua Condominium is a good choice ...


In [23]:
for i in range(10):
    print("Review #",i+1)
    print(reviews.location[i])
    for token in spans(reviews.location[i]):
        print(token)
        assert token[0]==reviews.location[i][token[1]:token[2]]
    

### Load location / ORG keywords
<a id='load_location_keywords'></a>

<a href='#declare_section'>Link to top</a>

In [23]:
#lode multiword location names (LOC)
word_locates = ['list_location.txt']
word_location = []
for word_locate in word_locates:
    with open(word_locate) as f:
        lines = f.readlines()
    word_location2 = [x.strip() for x in lines] 
    
    word_location.extend(word_location2)
word_location[0:5]

['99/89 Moo 6',
 'VIPHAVADEE RANDSIT RD',
 '111/1 Soi Ngam Wong Wan 47 Yaek 1',
 '310/72 Soi Phaholyothin 67/1',
 '3/2899 Soi Yak']

In [24]:
#load multiword hotel name (ORG)
word_hotels = ['list_hotel.txt']
lists_hotel = []
for word_hotel in word_hotels:
    with open(word_hotel) as f:
        lines = f.readlines()
    lists_hotel2 = [x.strip() for x in lines] 
    
    lists_hotel.extend(lists_hotel2)
lists_hotel[0:5]

['Petpimarn Boutique Resort',
 'Airport Suite Bangkok (Don Muang Airport)',
 'The Riche Residence',
 'Regent Home1 at Donmuang',
 'Charoenpong Apartment']

## Create training data for location/ORG
 <a id='train_location'>

In [73]:
new_data = []

f = IntProgress(min=0, max=len(reviews.location)) # instantiate the bar
display(f) # display the bar
for i in range(len(reviews.location)):
    list_tup = []
     
    f.value += 1 
    for token in spans(reviews.location[i]):
         
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        tuples = token[1],token[2]
        #print(token[1],token[2])
        tuples1 =  (token[1],token[2])
        if token[0] in word_location:
           
            tuples2 = tuples1 + (u"LOC",)          
            lis_lo3 =  tuples2
            list_tup.append(tuples2) 
        if token[0] in lists_hotel:
             
            tuples2 = tuples1 + (u"ORG",)          
            lis_lo3 =  tuples2
            list_tup.append(tuples2) 
    data_item = (reviews.location[i].decode('utf-8'), {"entities": list_tup})
    #print(data_item)
    new_data.append(data_item)
print(new_data[:5])

IntProgress(value=0, max=13019)

[(u'Staying at @ Home Executive Apartment is a good choice when you are visiting Central Pattaya.', {'entities': [(11, 37, u'ORG'), (77, 92, u'LOC')]}), (u'Staying at @ Oasis Resort is a good choice when you are visiting Wang Pong.', {'entities': [(11, 25, u'ORG'), (65, 74, u'LOC')]}), (u'Staying at @ T Boutique Hotel is a good choice when you are visiting Khlong Wan.', {'entities': [(11, 29, u'ORG'), (69, 79, u'LOC')]}), (u'Staying at AC Resort is a good choice when you are visiting Sairee beach.', {'entities': [(11, 20, u'ORG'), (60, 72, u'LOC')]}), (u'Staying at Acqua Condominium is a good choice when you are visiting Jomtien Beach.', {'entities': [(11, 28, u'ORG'), (68, 81, u'LOC')]})]


In [25]:
def save_data(filename,new_data):
    with open(filename,"w") as f:
        for item in new_data:
            text = item[0].encode('utf-8')
            enti_dict = item[1]
            #print(enti_dict
            list_ent = enti_dict['entities']
            
            f.write(text +"\n")
            f.write("######\n")
            for label_en in list_ent:
                f.write("%d,%d,%s,"%(label_en[0],label_en[1],label_en[2]))
            f.write("\n")
    f.close()
    
    
def load_data(filename):
    with open(filename,"r") as f:
        data = []
        all_lines  = f.readlines()
        i = 0
        f = IntProgress(min=0, max=len(all_lines)) # instantiate the bar
        display(f) # display the bar
        while (i < len(all_lines)):
            text  = all_lines[i]
            text2 = ''
            while (text2.strip() != '######'):
                i += 1
                f.value += 1
                text = text + text2
                text2  = all_lines[i] 
                
            i += 1
            f.value +=1
            mark_str = all_lines[i]
            mark_item = mark_str.split(',')
            j = 0
            tuple_ent_list = []
            #print(text)
            #print(mark_item)
            while (j < len(mark_item)):
                #print(mark_item[j])
                if mark_item[j] == '\n':
                    break
                tuple_ent_list.append((mark_item[j],mark_item[j+1],mark_item[j+2]))
                j += 3
            save_item = (text, {'entities':tuple_ent_list})
            data.append(save_item)
            i += 1
            f.value+= 1
        
        f.close()
        print("finish loading %d rows"%i)
    return data

In [126]:
save_data("location.train",new_data)


In [26]:
new_data2 = load_data("location.train")


IntProgress(value=0, max=47010)

In [27]:
new_data2[:5]
new_data = new_data2

In [74]:
random.shuffle(new_data)

train_data = new_data[:int(0.7*len(new_data))]
test_data = new_data[int(0.7*len(new_data)):]

### Begin pre-load model

In [31]:
# coding: utf8
"""Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

from __future__ import unicode_literals, print_function


In [76]:
# training data
TRAIN_DATA =  train_data
TEST_DATA = test_data

In [33]:
model = "en"
output_dir = "spacy_trained"
n_iter = 1000
 

In [34]:
"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

In [35]:
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")


In [36]:
if model is None:
    optimizer = nlp.begin_training()
else:
    # Note that 'begin_training' initializes the models, so it'll zero out
    # existing entity types.
    optimizer = nlp.entity.create_optimizer()

In [37]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])


###  Start traing ORG/LOC 
 

In [33]:
n_iter = 100

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    if model is None:
        nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print(itn," Losses", losses)

In [40]:
#Save model


from spacy import displacy
# save model to output directory
output_dir = "spacy_trained"
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

    

<a href='#declare_section'>Link to top</a>

### Load  location/ORG model
<a id='load_location'>

In [29]:
# test the saved model
output_dir = "spacy_trained"
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
#for text, _ in TRAIN_DATA:
    #doc = nlp2(text)
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        

        




Loading from spacy_trained


### Test LOC/ORG model  
<a id='test_location'>

In [77]:
TEST_DATA = test_data
print("Testing totally ", len(TEST_DATA))

Testing totally  3906


In [90]:
from spacy import displacy

TEST_DATA = test_data
# test the trained model
count = 0
total_predicted = 0
total_annote = 0

f = IntProgress(min=0, max=len(TEST_DATA)) # instantiate the bar
display(f) # display the bar

for text, annotations in TEST_DATA:
    doc = nlp2(text )
    f.value += 1
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    predicted_list = [(ent.text, ent.label_) for ent in doc.ents]
    total_predicted += len(predicted_list)
    #print("label", [ ent for ent in annotations.get("entities")])
    if count < 20:
        displacy.render(doc, jupyter=True, style='ent')
    #print(annotations)
    item_list = []
    for i in annotations['entities']:
         
        item =  {'start': int(i[0]), 'end': int(i[1]), 'label': i[2]}
        item_list.append(item)
    total_annote += len(item_list)
     
    my_ent =   {}
    my_ent['text'] = text
    my_ent['ents'] = item_list
    my_ent['title'] = None
    if count < 20:
        print("-----")
        displacy.render(my_ent, jupyter=True, style='ent',manual=True )
    count += 1

print("annote %d predicted %d"%(total_annote,total_predicted))
    

IntProgress(value=0, max=3906)

-----


-----


-----


-----


...

-----


-----


-----


-----


annote 19787 predicted 29873


In [91]:
from spacy import displacy

 
# test the trained model
count = 0
total_predicted = 0
total_annote = 0

f = IntProgress(min=0, max=len(TRAIN_DATA)) # instantiate the bar
display(f) # display the bar

for text, annotations in TRAIN_DATA:
    doc = nlp2(text )
    f.value += 1
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    predicted_list = [(ent.text, ent.label_) for ent in doc.ents]
    total_predicted += len(predicted_list)
    #print("label", [ ent for ent in annotations.get("entities")])
    if count < 20:
        displacy.render(doc, jupyter=True, style='ent')
    #print(annotations)
    item_list = []
    for i in annotations['entities']:
         
        item =  {'start': int(i[0]), 'end': int(i[1]), 'label': i[2]}
        item_list.append(item)
    total_annote += len(item_list)
     
    my_ent =   {}
    my_ent['text'] = text
    my_ent['ents'] = item_list
    my_ent['title'] = None
    if count < 20:
        print("-----")
        displacy.render(my_ent, jupyter=True, style='ent',manual=True )
    count += 1

print("annote %d predicted %d"%(total_annote,total_predicted))

IntProgress(value=0, max=9113)

-----


-----


-----


-----


...

-----


-----


-----


-----


annote 46327 predicted 70156


In [85]:
TRAIN_DATA = train_data
count_accuracy(TRAIN_DATA)

total 0/123/46327


(0.0, 0.0)

In [86]:
count_accuracy(TEST_DATA)

total 2/54/19787


(0.037037037037037035, 0.0001010764643452772)

<a href='#declare_section'>Link back to top</a>



***

# Train Facility  
<a id='train_facility'></a>


In [21]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
    "km" : "kilometers",
    "mi" : "miles"
}

In [22]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        # We are not using "text.split()" here
        #since it is not fool proof, e.g. words followed by punctuations "Are you kidding?I think you aren't."
        text = re.findall(r"[\w']+", text)
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)# remove links
    text = re.sub(r'\<a href', ' ', text)# remove html link tag
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.!?:#$\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

### Read facility corpus
<a id='load_facility_keywords'></a>
<a href='#declare_section'>Link to top</a>

In [23]:
reviews_2 = pd.read_csv("./facility.csv")
reviews_2 = reviews_2.dropna()
reviews_2 = reviews_2.reset_index(drop=True)

In [24]:
reviews_2.head()

,Facility
0,"For the comfort and convenience of guests,Bang..."
1,"For the comfort and convenience of guests,Baan..."
2,"For the comfort and convenience of guests,Siam..."
3,"For the comfort and convenience of guests,Yota..."
4,"For the comfort and convenience of guests,Chaw..."


In [25]:
reviews_2.Facility[:5]

0    For the comfort and convenience of guests,Bang...
1    For the comfort and convenience of guests,Baan...
2    For the comfort and convenience of guests,Siam...
3    For the comfort and convenience of guests,Yota...
4    For the comfort and convenience of guests,Chaw...
Name: Facility, dtype: object

In [26]:
clean_facilities = []
for facility in reviews_2.Facility:
    clean_facilities.append(clean_text(facility, remove_stopwords=False))
print("facility are complete.")

facility are complete.


In [27]:
clean_facilities[:5]

['for the comfort and convenience of guests bangkok test hotel offers desk hairdryer in room safe',
 'for the comfort and convenience of guests baan samrarn offers refrigerator tv shower',
 'for the comfort and convenience of guests siam golden place offers ac family room non smoking room',
 'for the comfort and convenience of guests yotaka boutique hotel bangkok offers shops atm banking hair salon',
 'for the comfort and convenience of guests chaweng inn offers atm banking beauty salon grocery']

In [28]:
new_data2 = []
 

f = IntProgress(min=0, max=len(reviews_2.Facility)) # instantiate the bar
display(f) # display the bar

for i in range(len(reviews_2.Facility)):
    list_tup = []
    f.value += 1
     
    for token in spans(reviews_2.Facility[i].decode('utf-8')):
         
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        tuples = token[1],token[2]
        
        tuples1 =  (token[1],token[2])
        #print(token[0],token[1],token[2]) 
        if token[0] in word_list_fac:
           
            tuples2 = tuples1 + (u"FACILITY",)          
            lis_lo3 =  tuples2
            #print(token[0],token[1],token[2])
            list_tup.append(tuples2) 
        
    data_item = (reviews_2.Facility[i].decode('utf-8'), {"entities": list_tup})
    #print(data_item)
    new_data2.append(data_item)
print(new_data2[:5])

IntProgress(value=0, max=13522)

[(u'For the comfort and convenience of guests,Bangkok Test Hotel offers Desk,Hairdryer,In-room safe', {'entities': [(68, 72, u'FACILITY'), (73, 82, u'FACILITY'), (83, 95, u'FACILITY')]}), (u'For the comfort and convenience of guests,Baan Samrarn offers Refrigerator,TV,Shower', {'entities': [(62, 74, u'FACILITY'), (75, 77, u'FACILITY'), (78, 84, u'FACILITY')]}), (u'For the comfort and convenience of guests,Siam Golden Place offers AC,Family room,Non-smoking room', {'entities': [(67, 69, u'FACILITY'), (70, 81, u'FACILITY'), (82, 98, u'FACILITY')]}), (u'For the comfort and convenience of guests,Yotaka Boutique Hotel Bangkok offers Shops,ATM/Banking,Hair salon', {'entities': [(79, 84, u'FACILITY'), (85, 96, u'FACILITY'), (97, 107, u'FACILITY')]}), (u'For the comfort and convenience of guests,Chaweng Inn offers ATM/Banking,Beauty salon,Grocery', {'entities': [(61, 72, u'FACILITY'), (73, 85, u'FACILITY'), (86, 93, u'FACILITY')]})]


### Train facility 
<a id='train_facility'>

In [103]:
new_data2[:5]

[(u'For the comfort and convenience of guests,Bangkok Test Hotel offers Desk,Hairdryer,In-room safe',
  {'entities': [(68, 72, u'FACILITY'),
    (73, 82, u'FACILITY'),
    (83, 95, u'FACILITY')]}),
 (u'For the comfort and convenience of guests,Baan Samrarn offers Refrigerator,TV,Shower',
  {'entities': [(62, 74, u'FACILITY'),
    (75, 77, u'FACILITY'),
    (78, 84, u'FACILITY')]}),
 (u'For the comfort and convenience of guests,Siam Golden Place offers AC,Family room,Non-smoking room',
  {'entities': [(67, 69, u'FACILITY'),
    (70, 81, u'FACILITY'),
    (82, 98, u'FACILITY')]}),
 (u'For the comfort and convenience of guests,Yotaka Boutique Hotel Bangkok offers Shops,ATM/Banking,Hair salon',
  {'entities': [(79, 84, u'FACILITY'),
    (85, 96, u'FACILITY'),
    (97, 107, u'FACILITY')]}),
 (u'For the comfort and convenience of guests,Chaweng Inn offers ATM/Banking,Beauty salon,Grocery',
  {'entities': [(61, 72, u'FACILITY'),
    (73, 85, u'FACILITY'),
    (86, 93, u'FACILITY')]})]

In [32]:
random.shuffle(new_data2)

train_data = new_data2[:int(0.7*len(new_data2))]
test_data = new_data2[int(0.7*len(new_data2)):]

TRAIN_DATA = train_data
TEST_DATA = test_data

In [29]:
model = "en"
output_dir = "spacy_trained_fac"
n_iter = 100
 

In [93]:
"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

In [94]:
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")


In [95]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        #print(ent[2])
        ner.add_label(ent[2])


In [98]:
if model is None:
    optimizer = nlp.begin_training()
else:
    # Note that 'begin_training' initializes the models, so it'll zero out
    # existing entity types.
    optimizer = nlp.entity.create_optimizer()

In [ ]:
n_iter  = 100
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                       losses=losses)
        print(itn,'Losses', losses)

In [ ]:
# save model to output directory
output_dir = "spacy_trained_fac"
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)


<a href='#declare_section'>Link to top</a>

### load model / Test
<a id='load_facility'>

In [30]:
from spacy import displacy
output_dir = "spacy_trained_fac"
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
#for text, _ in TEST_DATA:
#    doc = nlp2(text)
#    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    

Loading from spacy_trained_fac


### Test facility model
<a id='test_facility'></a>
<a href='#declare_section'>Link to top</a>

In [ ]:
count_correct(TRAIN_DATA)

In [ ]:
count_correct(TEST_DATA)

In [99]:
for text, annotations in TEST_DATA:
    doc = nlp2(text)
    for chunk in doc.noun_chunks:
        print(chunk.text, chunk.label_, chunk.root.text)


Lannathai Guesthouse NP Guesthouse
Chiang Mai NP Mai
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free High Speed Internet NP Internet
Service NP Service
Free Parking NP Parking
YHA Bangkok Airport Hostel NP Hostel
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free High Speed Internet NP Internet
WiFi),Airport Transportation NP Transportation
Dry Cleaning NP Cleaning
the comfort NP comfort
convenience NP convenience
guests NP guests
The Greens 8 Rawai 2 Bedrooms Villa NP Villa
Multilingual staff NP staff
Tours NP Tours
Parking NP Parking
the comfort NP comfort
convenience NP convenience
guests NP guests
Dusit Thani Bangkok NP Bangkok
A la carte breakfast NP breakfast
A la carte dinner NP dinner
A la carte lunch NP lunch
Aonang Paradise Resort NP Resort
Krabi/Ao Nang NP Nang
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Pool

The Palm Delight Lodge NP Lodge
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
WiFi NP WiFi
public area NP area
Coffee shop NP shop
TV NP TV
the comfort NP comfort
convenience NP convenience
guests NP guests
Shangri-La Hotel Bangkok NP Bangkok
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Fitness Center NP Center
Gym / Workout Room NP Room
Pool NP Pool
Room Service NP Service
VR Hostel NP Hostel
Bangkok NP Bangkok
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free Parking NP Parking
Free High Speed Internet NP Internet
(WiFi),Non-Smoking Hotel NP Hotel
U Rip Resort Specialty Hotel NP Hotel
Ko Phi Phi NP Phi
Don NP Don
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Pool NP Pool
Free High Speed Internet NP Internet
Sea Breeze Resort NP Resort
Ko Pha Ngan NP 

the comfort NP comfort
convenience NP convenience
guests NP guests
Promsook Apartment NP Apartment
Desk NP Desk
Refrigerator NP Refrigerator
Shower NP Shower
Visit Natural Detox Resort Hotel NP Hotel
Ko Samui/Lamai Beach NP Beach
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Pool NP Pool
Free Parking NP Parking
Restaurant NP Restaurant
the comfort NP comfort
convenience NP convenience
guests NP guests
iPREMIUM Sukhumvit NP Sukhumvit
24-hour room service NP service
Room service NP service
Parking NP Parking
Absolute Sanctuary Specialty Hotel NP Hotel
Ko Samui NP Samui
Bophut NP Bophut
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Pool NP Pool
Restaurant NP Restaurant
Room Service NP Service
Free Parking NP Parking
Hop Inn Chiang Mai Hotel NP Hotel
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free High Speed Internet NP Inter

Kanda Residences Resort NP Resort
Ko Samui NP Samui
Bophut NP Bophut
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Pool NP Pool
Room Service NP Service
Free Parking NP Parking
Restaurant NP Restaurant
The Sukosol Hotel NP Hotel
Bangkok NP Bangkok
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Pool NP Pool
Room Service NP Service
Restaurant NP Restaurant
Fitness Center NP Center
Gym / Workout Room NP Room
the comfort NP comfort
convenience NP convenience
guests NP guests
Somerset Park Suanplu Bangkok NP Bangkok
Children club NP club
Children club NP club
Children NP Children
Costa Village Pattaya NP Pattaya
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Pool NP Pool
Free Parking NP Parking
Restaurant NP Restaurant
Nitchanan Villa NP Villa
Ko Pha Ngan NP Ngan
the comfort NP comfort
your own home NP home
The property NP property
fantastic fac

Moonlight Cottage Hotel NP Hotel
Phuket NP Phuket
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free Parking NP Parking
Restaurant NP Restaurant
,Free High Speed Internet NP Internet
(WiFi NP WiFi
the comfort NP comfort
convenience NP convenience
guests NP guests
Train Guesthouse NP Guesthouse
Desk NP Desk
-room NP room
Silom Art Hostel NP Hostel
Bangkok NP Bangkok
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free High Speed Internet NP Internet
(WiFi),Non-Smoking Hotel NP Hotel
Public Wifi NP Wifi
Dream Cat-Cher Hostel NP Hostel
Ko Samui NP Samui
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free High Speed Internet NP Internet
Parking,Multilingual Staff NP Staff
Bangkok NP Bangkok
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free Parking NP Parking
R

...

the comfort NP comfort
convenience NP convenience
guests NP guests
Khaosok Boutique Camps NP Camps
Parking NP Parking
Coffee/tea NP tea
lobby NP lobby
the comfort NP comfort
convenience NP convenience
guests NP guests
Sangthong Resort NP Resort
Coffee shop NP shop
Parking NP Parking
Restaurant NP Restaurant
SEA Harmony Eco Lodge Guest house NP house
Hua Hin NP Hin
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Restaurant NP Restaurant
Free High Speed Internet NP Internet
WiFi),Breakfast NP WiFi),Breakfast
Free Parking NP Parking
the comfort NP comfort
convenience NP convenience
guests NP guests
a Box Hostel NP Hostel
Shops NP Shops
ATM NP ATM
Banking NP Banking
Hair NP Hair
Kirikayan Luxury Pool Villas NP Villas
Spa Hotel NP Hotel
Ko Samui NP Samui
Thailand NP Thailand
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Pool NP Pool
Room Service NP Service
Restaurant NP Restaurant
Free Parking NP Parki

Bamboo 99 Old City Guesthouse NP Guesthouse
Chiang Mai NP Mai
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Room Service NP Service
Restaurant NP Restaurant
,Bar/Lounge NP Lounge
Spa NP Spa
Casuarina Shores Condominium NP Condominium
Phuket/Bang Tao Beach NP Beach
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Pool NP Pool
Free Parking NP Parking
Free High Speed Internet NP Internet
(WiFi NP WiFi
the comfort NP comfort
convenience NP convenience
guests NP guests
Orange Tree House NP House
Room service NP service
Parking NP Parking
WiFi NP WiFi
public area NP area
Lanta Nice Beach House Lodge NP Lodge
Ko Lanta NP Lanta
the comfort NP comfort
your own home NP home
The property NP property
fantastic facilities NP facilities
Free Parking NP Parking
Free High Speed Internet NP Internet
Service NP Service
Lanta Resort NP Resort
Ko Lanta NP Lanta
the comfort NP comfort
your own home NP home

VPlace Silom Hotel NP Hotel
Bangkok NP Bangkok
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Room Service NP Service
Restaurant,Free High Speed Internet (WiFi NP WiFi
the comfort NP comfort
convenience NP convenience
guests NP guests
Lek City Hotel NP Hotel
Shops NP Shops
ATM NP ATM
Banking NP Banking
Hair NP Hair
the comfort NP comfort
convenience NP convenience
guests NP guests
Arun Residence NP Residence
Cable TV NP TV
Hairdryer NP Hairdryer
In-room NP room
Bamboo House Phuket Boutique Hotel NP Hotel
Karon NP Karon
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Pool NP Pool
Restaurant NP Restaurant
Free High Speed Internet NP Internet
WiFi),Airport Transportation NP Transportation
the comfort NP comfort
convenience NP convenience
guests NP guests
Royal Heritage Pavilion Jomtien Boutique Resort NP Resort
Parking,Safety deposit box NP box
WiFi NP WiFi
public area NP area
The Peaberry Hotel NP Ho

Ammatara Pura Pool Villa NP Villa
Ko Samui NP Samui
Lamai Beach NP Beach
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Pool NP Pool
Room Service NP Service
Free Parking NP Parking
Restaurant NP Restaurant
the comfort NP comfort
convenience NP convenience
guests NP guests
Ma Maison NP Maison
Coffee shop NP shop
Restaurant,24-hour Receptionist NP Receptionist
the comfort NP comfort
convenience NP convenience
guests NP guests
Sandalay Resort Pattaya NP Pattaya
Parking NP Parking
Elevator,24-hour room service NP service
the comfort NP comfort
convenience NP convenience
guests NP guests
Koh Jum Hostel NP Hostel
Concierge NP Concierge
Front desk NP desk
Tours NP Tours
Suksawad Hotel NP Hotel
Bangkok NP Bangkok
plenty NP plenty
room choices NP choices
all tastes NP tastes
many facilities NP facilities
Room Service NP Service
Restaurant NP Restaurant
,Free High Speed Internet NP Internet
(WiFi),Concierge NP WiFi),Concierge
Good Hostel NP Hostel
the

KeyboardInterrupt: 

In [33]:
# test the   model
count = 0
total_predicted = 0
total_annote = 0

f = IntProgress(min=0, max=len(TEST_DATA)) # instantiate the bar
display(f) # display the bar

for text, annotations in TEST_DATA:
    doc = nlp2(text)
    count += 1
    f.value += 1
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    predicted_list = [(ent.text, ent.label_) for ent in doc.ents]
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    if count < 20:
        displacy.render(nlp2(str(text).decode('utf-8')), jupyter=True, style='ent')
    
    item_list = []
    for i in annotations['entities']:
         
        item =  {'start': int( i[0]), 'end': int(i[1]), 'label': i[2]}
        item_list.append(item)
    total_annote += len(item_list)
    total_predicted += len(predicted_list)
    my_ent =   {}
    my_ent['text'] = text
    my_ent['ents'] = item_list
    my_ent['title'] = None
    if (count < 20) :
        print("-----")
        displacy.render(my_ent, jupyter=True, style='ent',manual=True )
print("%d %d"%(total_annote,total_predicted))


IntProgress(value=0, max=4057)

-----


-----


-----


-----


...

-----


-----


-----


-----


9427 7999


In [ ]:
print("Testing total:",len(TEST_DATA))

In [34]:
# test the   model
count = 0
total_predicted = 0
total_annote = 0

f = IntProgress(min=0, max=len(TRAIN_DATA)) # instantiate the bar
display(f) # display the bar

for text, annotations in TRAIN_DATA:
    doc = nlp2(text)
    count += 1
    f.value += 1
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    predicted_list = [(ent.text, ent.label_) for ent in doc.ents]
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    if count < 20:
        displacy.render(nlp2(str(text).decode('utf-8')), jupyter=True, style='ent')
    
    item_list = []
    for i in annotations['entities']:
         
        item =  {'start': int( i[0]), 'end': int(i[1]), 'label': i[2]}
        item_list.append(item)
    total_annote += len(item_list)
    total_predicted += len(predicted_list)
    my_ent =   {}
    my_ent['text'] = text
    my_ent['ents'] = item_list
    my_ent['title'] = None
    if (count < 20) :
        print("-----")
        displacy.render(my_ent, jupyter=True, style='ent',manual=True )
print("%d %d"%(total_annote,total_predicted))

IntProgress(value=0, max=9465)

-----


-----


-----


-----


...

-----


-----


-----


-----


22167 18716


<a href='#declare_section'>Link to top</a>
***

# Train ORG/LOC/FAC
<a id='load_both'></a>

In [112]:
reviews_3 = pd.read_csv("./all_loc_fac.csv")
reviews_3 = reviews_3.dropna()
reviews_3 = reviews_3.reset_index(drop=True)

In [113]:
reviews_3.head()

,location
0,Staying at @ Home Executive Apartment is a goo...
1,Staying at @ Oasis Resort is a good choice whe...
2,Staying at @ T Boutique Hotel is a good choice...
3,Staying at AC Resort is a good choice when you...
4,Staying at Acqua Condominium is a good choice ...


In [114]:
import numpy as np
np.random.shuffle(reviews_3.values)
 

In [115]:
from ipywidgets import IntProgress
from IPython.display import display

In [121]:
import tqdm

new_data3 = []
line_num=0

f = IntProgress(min=0, max=len(reviews_3.location)) # instantiate the bar
display(f) # display the bar

for i in range(len(reviews_3.location)):
#for i in tqdm.tqdm(range(len(reviews_3.location))):
    list_tup = []
     
    #line_num += 1
    f.value += 1 
    for token in spans(reviews_3.location[i].decode('utf-8')):
         
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        tuples = token[1],token[2]
        #print(token[1],token[2])
        tuples1 =  (token[1],token[2])
        if token[0] in word_location:
           
            tuples2 = tuples1 + (u"LOC",)          
            lis_lo3 =  tuples2
            
            list_tup.append(tuples2) 
        if token[0] in lists_hotel:
             
            tuples2 = tuples1 + (u"ORG",)          
            lis_lo3 =  tuples2
            list_tup.append(tuples2) 
            
        if token[0] in word_list_fac:
           
            tuples2 = tuples1 + (u"FACILITY",)          
            lis_lo3 =  tuples2
            #print(token[0],token[1],token[2])
            list_tup.append(tuples2) 
            
    data_item = (reviews_3.location[i].decode('utf-8'), {"entities": list_tup})
    #print(data_item)
    line_num += 1
    new_data3.append(data_item)
 
print(new_data3[:5])

IntProgress(value=0, max=26541)

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:21: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/usr/lib/python2.7/site-packages/ipykernel_launcher.py:27: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


[(u'For the comfort and convenience of guests,Boss Suites Pattaya offers Laundry service,Tours,Luggage storage', {'entities': [(42, 61, u'ORG'), (69, 84, u'FACILITY'), (85, 90, u'FACILITY'), (91, 106, u'FACILITY')]}), (u'Happio Hotel Lodge, Bangkok has plenty of room choices to suit all tastes, and many facilities such as Restaurant,Bar/Lounge,Free High Speed Internet (WiFi),Concierge', {'entities': [(0, 18, u'ORG'), (20, 27, u'LOC'), (103, 113, u'FACILITY'), (151, 155, u'FACILITY'), (157, 166, u'FACILITY')]}), (u'with you stay at Nicha Residence Hotel, Kamala, Phuket The hotel has a very good location, also near Thong Dee The Kathu Brasserie,Cafe de Paris,Vista,Saffron Spice Restaurant & Bar,Bangla Road,Patong Beach,Exclusive Tailor,Magnifique Tailor Phuket Intl Airport,Krabi Airport', {'entities': [(17, 38, u'ORG'), (40, 46, u'LOC'), (48, 54, u'LOC'), (115, 120, u'LOC'), (131, 135, u'FACILITY'), (145, 150, u'LOC'), (165, 175, u'FACILITY'), (178, 181, u'FACILITY'), (189, 193, u'LOC'),

In [125]:
random.shuffle(new_data3)

train_data = new_data3[:int(0.7*len(new_data3))]
test_data = new_data3[int(0.7*len(new_data3)):]

TRAIN_DATA = train_data
TEST_DATA = test_data

### Train all
<a id='train_both'></a>

In [47]:
model = "en"
output_dir = "spacy_trained_loc_fac"
n_iter = 100 

In [ ]:
"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

In [ ]:
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")

In [ ]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])


In [ ]:
import tqdm

n_iter = 100

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    if model is None:
        nlp.begin_training()
    for itn in tqdm.tqdm(range(n_iter)):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print(itn," Losses", losses)

In [ ]:
# save model to output directory
output_dir = "spacy_trained_loc_fac"
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)


<a href='#declare_section'>Link to top</a>

### Load saved model (LOC/ORG/FAC)
<a id='load_both'></a>


In [122]:
from spacy import displacy

output_dir = "spacy_trained_loc_fac"
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

from spacy import displacy
       
#for text, _ in TEST_DATA:
#    doc = nlp2(text)
#    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        

Loading from spacy_trained_loc_fac


### Test model
<a id='test_both'></a>

In [130]:
# test the trained model use test data
import tqdm

from ipywidgets import FloatProgress
from IPython.display import display

f = FloatProgress(min=0, max=len(TEST_DATA))
display(f)

total_predicted = 0
total_annote = 0
count = 0

count_ent_type_pred =  dict()
count_ent_type_pred['label'] = 'predicted'
count_ent_type =  dict()
count_ent_type ['label'] = 'annotate'    
#f = IntProgress(min=0, max=len(TEST_DATA)) # instantiate the bar
#display(f) # display the bar

print("Testing totally ", len(TEST_DATA))
for text, annotations in (TEST_DATA):
#for text, annotations in tqdm.tqdm(TEST_DATA):

    doc = nlp2(text)
    f.value += 1
    
    
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    predicted = [(ent.text, ent.label_) for ent in doc.ents]
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    if count < 20:
        displacy.render(nlp2(text ), jupyter=True, style='ent')
    
    item_list = []
    for i in annotations['entities']:
         
        item =  {'start': int(i[0]), 'end': int(i[1]), 'label': i[2]}
        item_list.append(item)
    total_predicted += len(predicted) 
    total_annote += len(item_list)
    
    for ele in predicted:         
        count_ent_type_pred[ele[1]] = count_ent_type_pred.get(ele[1],0) + 1
      
    #print(count_ent_type_pred)
    #print(count_ent_type)
    for ele in annotations['entities']:        
        count_ent_type [ele[2]] = count_ent_type.get(ele[2],0) + 1
        
   
    my_ent =   {}
    my_ent['text'] = text
    my_ent['ents'] = item_list
    my_ent['title'] = None
     
    if count < 20:
        print("-----")
        displacy.render(my_ent, jupyter=True, style='ent',manual=True )
    count += 1
print("%d / %d\n"%(total_annote,total_predicted))
print(count_ent_type)
print(count_ent_type_pred)


FloatProgress(value=0.0, max=7963.0)

Testing totally  7963


-----


-----


-----


...

-----


-----


-----


-----


40745 / 37387

{u'LOC': 17088, u'ORG': 8890, u'FACILITY': 14767, 'label': 'annotate'}
{u'LOC': 15351, u'ORG': 8711, u'FACILITY': 13324, u'CARDINAL': 1, 'label': 'predicted'}


In [132]:
# test the trained model for train data
import tqdm

pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=len(TRAIN_DATA)).start()


#f = FloatProgress(min=0, max=len(TRAIN_DATA))
#display(f)

total_predicted = 0
total_annote = 0
count = 0
#f = IntProgress(min=0, max=len(TEST_DATA)) # instantiate the bar
#display(f) # display the bar

count_ent_type_pred =  dict()
count_ent_type_pred['label'] = 'predicted'
count_ent_type =  dict()
count_ent_type ['label'] = 'annotate'    

print("Testing totally ", len(TRAIN_DATA))
#for text, annotations in tqdm.tqdm(TRAIN_DATA):
for text, annotations in TRAIN_DATA:
    doc = nlp2(text)
    #f.value += 1
    count += 1
    
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    predicted = [(ent.text, ent.label_) for ent in doc.ents]
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    if count < 20:
        displacy.render(nlp2(text ), jupyter=True, style='ent')
    
    item_list = []
    for i in annotations['entities']:
         
        item =  {'start': int(i[0]), 'end': int(i[1]), 'label': i[2]}
        item_list.append(item)
    total_predicted += len(predicted) 
    total_annote += len(item_list)
     
    for ele in predicted:         
        count_ent_type_pred[ele[1]] = count_ent_type_pred.get(ele[1],0) + 1
      
     
  
    for ele in annotations['entities']:        
        count_ent_type [ele[2]] = count_ent_type.get(ele[2],0) + 1
        
   
    my_ent =   {}
    my_ent['text'] = text
    my_ent['ents'] = item_list
    my_ent['title'] = None
     
    if count < 20:
        print("-----")
        displacy.render(my_ent, jupyter=True, style='ent',manual=True )
    pbar.update(count)
print("%d / %d\n"%(total_annote,total_predicted))
print(count_ent_type)
print(count_ent_type_pred)


Testing totally  18578


-----


-----


-----


-----


...

-----


-----


-----


-----


93661 / 85867

{u'LOC': 39034, u'ORG': 20747, u'FACILITY': 33880, 'label': 'annotate'}
{u'LOC': 34865, u'ORG': 20416, u'FACILITY': 30585, u'CARDINAL': 1, 'label': 'predicted'}


In [133]:
save_data("all.train",new_data3)

<a href='#declare_section'>Link to top</a>
***

# Test with full des
<a id='load_desc'></a>

In [134]:
reviews_4 = pd.read_csv("./allProviceTra.csv")
reviews_4 = reviews_4.dropna()
reviews_4 = reviews_4.reset_index(drop=True)

In [294]:
reviews_4.head()

In [295]:
np.random.shuffle(reviews_4.values)
 

In [136]:
desc = [ i.strip() for i in reviews_4['description'] ]

desc = [  re.sub(r"([\n]*,[\n]+|[\n])",'', i.decode('utf-8')) for i in desc]
desc

[u"Miracle Grand Convention Hotel is located in area / city Lak Si.,The hotel has a very good location, also near the Don Mueang International Airport (DMK), which is only 5.21 km away.,The hotel is located only 8.58 km away from Mo Chit BTS Station.There are plenty of tourist attractions nearby, such as Chulabhorn Research Institute within 0.23 km, and Thung Song Hong Police Station within 1.06 km.Whether you are planning an event or other special occasions, Miracle Grand Convention Hotel is a great choice for you with a large and well-equipped function room to suit your requirements.Miracle Grand Convention Hotel is a hotel near Airport, an ideal accommodation while waiting for your next flight. Enjoy a satisfying place to rest during your transit.The highest quality service accompanying its extensive facilities will make you get the ultimate holiday experience.,The hotel\u2019s fitness center is a must-try during your stay here.,Have an enjoyable and relaxing day at the pool, whethe

In [307]:
from spacy import displacy

# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

### Test full description
<a id='test_desc'></a>

In [138]:
TEST_DATA = desc
# test the trained model
k = 0
for text in TEST_DATA:
    doc = nlp2(text)
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    if (k < 10):
         displacy.render(doc, jupyter=True, style='ent')
    else:
        break
    k += 1
    

<a href='#declare_section'>Link to top</a>
***